In [ ]:
import gymnasium as gym
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=False)
env.reset()


In [2]:
import numpy as np
def policy_unif(s):
  a = env.action_space.sample()
  return a
def best_policy(s):
  policy_map = {
    0: 1, 1: 2, 2: 1, 3: 0,
    4: 1, 5: 1, 6: 1, 7: 2,
    8: 2, 9: 2, 10: 1, 11: 2,
    12: 1, 13: 2, 14: 2, 15: 2
  }
  return policy_map.get(s, env.action_space.sample())

def best_policy_rand(s):
  random_value = np.random.rand()
  if random_value < 0.9:
    return best_policy(s)
  else:
    return policy_unif(s)

In [ ]:
observation, reward, terminated, truncated, info = env.step(env.action_space.sample())
print(observation)
print(reward)
print(terminated)
print(truncated)
print(info)

In [4]:
def collect_trajectory(policy):
  s0 = env.reset()[0]
  traj_list = [s0]
  while True:
    a0 = policy(s0)
    traj_list.append(a0)
    s1, r0, terminated, truncated, _ = env.step(a0)
    traj_list.append(r0)
    traj_list.append(s1)
    s0 = s1
    if terminated or truncated:
      break
  return traj_list[:-1] #removing the terminal state

def collect_trajectory_s_a(policy,s0,a0):
  env.reset()
  env.unwrapped.s = s0
  traj_list = [s0,a0]
  s1, r0, terminated, truncated, _ = env.step(a0)
  traj_list.append(r0)
  traj_list.append(s1)
  s0=s1
  if terminated or truncated:
    return traj_list[:-1]
  
  while True:
    a0 = policy(s0)
    traj_list.append(a0)
    s1, r0, terminated, truncated, _ = env.step(a0)
    traj_list.append(r0)
    traj_list.append(s1)
    s0 = s1
    if terminated or truncated:
      break
  return traj_list[:-1] #removing the terminal state

def compute_return(traj,gamma=0.99):
  if traj==[]:
    return 0
  else:
    return traj[2]+gamma*compute_return(traj[3:],gamma)
  
n_states = env.observation_space.n
n_actions = env.action_space.n

def policy_evaluation(policy):
    Q = np.zeros((n_states, n_actions))
    for state in range(n_states):
        for action in range(n_actions):
            traj = collect_trajectory_s_a(policy, state, action)
            Q[state, action] = compute_return(traj)
    return Q

In [ ]:
traj = collect_trajectory(best_policy_rand)
print(traj)
print(compute_return(traj))

In [ ]:
traj = collect_trajectory_s_a(best_policy,14,1)
print(traj)
print(compute_return(traj))

In [ ]:
policy_evaluation(best_policy)

In [ ]:
env.reset()


In [ ]:
s1, r0, terminated, truncated, _ = env.step(2)
print(s1)
print(env.state)